In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import copy
import re
import sys
from tqdm import tqdm
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sarahabdelazim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sarahabdelazim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
fake = pd.read_csv("data/raw/Fake.csv")
real = pd.read_csv("data/raw/true.csv")
fake['label'] = 'fake'
real['label'] = 'real'

In [3]:
#Strip text columns from whitespace
cols = ['title', 'text', 'date']
real[cols] = real[cols].apply(lambda x: x.str.strip())
fake[cols] = fake[cols].apply(lambda x: x.str.strip())

In [4]:
# Remove unecessary text from the beginning of the text column in real df.
real['text'] = real['text'].str.replace('^(.*?)?\s[-]', '', regex=True)
real['text'].head(10)

0     The head of a conservative Republican faction...
1     Transgender people will be allowed for the fi...
2     The special counsel investigation of links be...
3     Trump campaign adviser George Papadopoulos to...
4     President Donald Trump called on the U.S. Pos...
5     The White House said on Friday it was set to ...
6     President Donald Trump said on Thursday he be...
7     While the Fake News loves to talk about my so...
8     Together, we are MAKING AMERICA GREAT AGAIN! ...
9     Alabama Secretary of State John Merrill said ...
Name: text, dtype: object

In [5]:
#real['text'].iloc[1151]

In [6]:
# Combining datasets
all_news = pd.concat([real, fake])
all_news = all_news.replace(r'^\s*$', np.nan, regex=True)
all_news = all_news.dropna(subset=['text'])
all_news.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [7]:
# Cleaning date column

match_condn = r'\b([0-9])\b'
replace_str = r'0\1'
all_news['date'] = all_news['date'].str.replace(match_condn, replace_str, regex=True)

replacement = {
    "^Jan\s": "January ",
    "^Feb\s": "February ",
    "^Mar\s": "March ",
    "^Apr\s": "April ",
    "^Jun\s": "June ",
    "^Jul\s": "July ",
    "^Aug\s": "August ",
    "^Sep\s": "September ",
    "^Oct\s": "October ",
    "^Nov\s": "November ",
    "^Dec\s": "December ",
    "14[-]Feb[-]18$": "February 14, 2018",
    "15[-]Feb[-]18$": "February 15, 2018",
    "16[-]Feb[-]18$": "February 16, 2018",
    "17[-]Feb[-]18$": "February 17, 2018",
    "18[-]Feb[-]18$": "February 18, 2018",
    "19[-]Feb[-]18$": "February 19, 2018",
}

all_news['date'] = all_news['date'].replace(replacement, regex=True)

# Converting date column to datetime
all_news['date'] = pd.to_datetime(all_news['date'], format='%B %d, %Y', errors='coerce')
all_news.isna().sum() #10 columns are empty because they were not dates in the first place

title       0
text        0
subject     0
date       10
label       0
dtype: int64

In [8]:
# Limiting categories of Subjects to only 2 (politicsNews and worldnews)
all_news.query("subject == 'left-news'")

subject_replace = {'News':'other', 
                   'politics':'politicsNews', 
                   'left-news' : 'other',
                   'Government News': 'politicsNews',
                   'US_News': 'politicsNews',
                   'Middle-east': 'worldnews' 
            }

all_news = all_news.replace({"subject": subject_replace})

#Test df to check subjects have been replaced
all_news.query("subject == 'politics'")

,title,text,subject,date,label


In [9]:
all_news["title_text"] = all_news["title"].combine_first(all_news["text"])
all_news["title_text"] = all_news["title"] +" "+ all_news["text"]
all_news = all_news.drop(['title', 'text'], axis=1)
cols = all_news.columns.tolist()
cols = cols[-1:] + cols[:-1]
all_news = all_news[cols]
all_news

,title_text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",politicsNews,2017-12-31,real
1,U.S. military to accept transgender recruits o...,politicsNews,2017-12-29,real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,politicsNews,2017-12-31,real
3,FBI Russia probe helped by Australian diplomat...,politicsNews,2017-12-30,real
4,Trump wants Postal Service to charge 'much mor...,politicsNews,2017-12-29,real
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,worldnews,2016-01-16,fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,worldnews,2016-01-16,fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,worldnews,2016-01-15,fake
23479,How to Blow $700 Million: Al Jazeera America F...,worldnews,2016-01-14,fake


In [ ]:
# Feature Engineering

## Get number of special characters:


In [17]:
#all_news.to_csv("data/processed/all_news.csv", index=False)